# Lab 05: Motion correction and slice timing

The purpose of this lab is to practice correcting for volume-to-volume motion in a functional image series, and to learn about common correction parameters and how to describe them with numbers and figures. We will also practice slice-timing correction, and test its effect on the timeseries data. 

## Preparation
* Import python tools for plotting
* Copy functional image from lab 01 into our working directory

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nilearn import plotting
from nilearn import image

In [3]:
%%bash

cp ../01-Lab/osfshare/sub-97_task-flanker_bold.nii.gz my_bold.nii.gz

## Motion correction tools in FSL

mcflirt is the basic motion correction tool provided by FSL
* [User guide](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/MCFLIRT)

FSL also offers a tool to both detect motion outliers and run motion correction with mcflirt
* [FSL Motion Outliers](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLMotionOutliers)
* This will create a dummy variable that codes for volumes possibly corrupted by motion, which can be used to ignore those volumes in our functional image analysis later. 

In [4]:
%%bash

# Running the bare bones command like below will output only the confound matrix 
# The default metric for detecting outliers is refrms (root mean square intensity diffs with referece vol, which is middle volume)
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion 

In [6]:
%%bash
cat my_bold_motion

0   0   0   0   0   0   0   0   0   0   0   0  
1   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   0   0  
0   0   0   0   0   0   0   0   0   0   

In [44]:
confound_mat=pd.read_table('my_bold_motion',header=None)

# How to plot like a design matrix output in the denoise py script?

In [14]:
%%bash

# We can produce figures of the motion by adding the -p flag
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion -s refrms_ts -p refrms_plot

![refrms](refrms_plot.png)

In [13]:
%%bash

# This version of the command will use fd as the metric for detecting outliers
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion --fd -s fd_ts -p fd_plot

![fd](fd_plot.png)

In [9]:
%%bash

# This version of the command will use dvars as the metric for detecting outliers
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion --dvars -s dvars_ts -p dvars_plot

![dvars](dvars_plot.png)

In [30]:
dvars=pd.read_table('refrms_ts',header=None)

In [37]:
# Preview top values
dvars.head()

,0
0,0.000000
1,0.010767
2,0.000782
3,0.001384
4,0.001283


In [38]:
# Descriptives
dvars.describe()

,0
count,315.000000
mean,0.002674
std,0.002771
min,0.000000
25%,0.000982
50%,0.002067
75%,0.003539
max,0.021370


In [39]:
dvars.mean()

0    0.002674
dtype: float64

### Practice: what is the mean fd ?

In [40]:
# Python code
fd=pd.read_table('fd_ts',header=None)
fd.describe()

,0
count,315.000000
mean,0.101543
std,0.063065
min,0.000000
25%,0.064398
50%,0.099239
75%,0.125806
max,0.687702
